In [1]:
import os
import ctypes
from gtimer import GlobalTimer
from numpy.ctypeslib import ndpointer
import numpy as np

PLAN_LIB_PATH = "../cmake-build-debug/libmoveit_plan_compact.so"

clib = ctypes.cdll.LoadLibrary(PLAN_LIB_PATH)
MAX_STR_LEN = clib.get_max_str_len()
MAX_NAME_LEN = clib.get_max_name_len()
MAX_JOINT_NUM = clib.get_max_joint_num()
MAX_TRAJ_LEN = clib.get_max_traj_len()

class c_string(ctypes.Structure):
    _fields_ = [("buffer", ctypes.c_char*MAX_STR_LEN),
                ("len", ctypes.c_int)
                ]

class c_trajectory(ctypes.Structure):
    _fields_ = [("names_flt", ctypes.c_char*(MAX_JOINT_NUM*MAX_NAME_LEN)),
                ("joints", ctypes.c_double*(MAX_JOINT_NUM*MAX_TRAJ_LEN)),
                ("name_len", ctypes.c_int),
                ("joint_count", ctypes.c_int),
                ("joint_max", ctypes.c_int),
                ("traj_len", ctypes.c_int),
                ("success", ctypes.c_bool)
                ]

class c_plan_goal(ctypes.Structure):
    _fields_ = [("group_name", ctypes.c_char*MAX_NAME_LEN),
                ("tool_link", ctypes.c_char*MAX_NAME_LEN),
                ("goal_link", ctypes.c_char*MAX_NAME_LEN),
                ("goal_pose", ctypes.c_double*7)
                ]

clib.hello_cstr.restype = c_string
clib.hello_char.restype = ndpointer(dtype=ctypes.c_char, shape=(MAX_STR_LEN,))
clib.plan_compact.restype = c_trajectory
clib.plan_compact.argtypes = [c_plan_goal]

def convert_trajectory(traj):
    names = traj.names_flt.split()
    joints = []
    for i_traj in range(traj.traj_len):
        joints.append(traj.joints[i_traj*traj.joint_max:i_traj*traj.joint_max+traj.joint_count])
    return names, np.array(joints), traj.success


In [2]:
gtimer = GlobalTimer.instance()

In [3]:
urdf_path = "../test_assets/custom_robots.urdf"
srdf_path = "../test_assets/custom_robots.srdf"

urdf_str = c_string()
srdf_str = c_string()
with open(urdf_path, 'r') as f:
    while True:
        line = f.readline()
        urdf_str.buffer += line
        if not line: break
            
urdf_str.buffer
with open(srdf_path, 'r') as f:
    while True:
        line = f.readline()
        srdf_str.buffer += line
        if not line: break

In [4]:
gtimer.reset(1e3,'ms')
gtimer.tic("init_ros")
clib.init_planner(urdf_str, srdf_str)
gtimer.toc("init_ros")

94.99597549438477

In [5]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_goal()
    goal.group_name = "indy0"
    goal.tool_link = "indy0_tcp"
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    goal.goal_link = "base_link"
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    names, traj, succ = convert_trajectory(c_traj)
    gtimer.toc("convert_trajectory")
print(gtimer)

init_ros: 	95.0 ms/1 = 94.996 ms (94.996/94.996)
plan_compact: 	242.0 ms/10 = 24.208 ms (18.136/39.187)
convert_trajectory: 	0.0 ms/10 = 0.03 ms (0.027/0.044)

